# Machine Learning Regression und Klassifikation Vertiefung

## TEIL A: Regression mit Hauspreisberechnung

<div style="padding: 5px; border: 5px solid #a10000ff;">

**Hinweis:** In den Codezellen sind jeweils einige Codeteile nicht programmiert. Diesen Code müssen Sie ergänzen. Die jeweiligen Stellen sind mit einem Kommentar und dem Keyword **TODO** vermerkt und z.T. Stellen mit ... markiert.

Ausserdem gibt es einige assert Statements. Diese geben einen Fehler aus, sollte etwas bei Ihrer Programmierung nicht korrekt sein.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests
import os
from sklearn.metrics import ConfusionMatrixDisplay

In diesem Dataset wurden verschiedene Eigenschaften von Liegenschaften erfasst. 

Dabei soll nun von den Eigenschaften auf den Hauspreis geschlossen werden. Der Hauspreis ist somit die **Zielvariable** oder engl. *Target*, ähnlich dem Label in der Klassifikation.

Die Berechnungen des Hauspreises, werden wir mit einem Regressionsmodell machen.

Das Dataset das wir benutzten, ist das California Housing Dataset:
https://media.geeksforgeeks.org/wp-content/uploads/20240522145850/housing%5B1%5D.csv

**Führen Sie die nächsten zwei Zellen aus**

In [ ]:
#check if file housing.csv exists, if not download it
if not os.path.exists("./data/housing.csv"):
    #load housing dataset from URL and save file locally
    url = "https://media.geeksforgeeks.org/wp-content/uploads/20240522145850/housing%5B1%5D.csv"
    response = requests.get(url)
    with open("./data/housing.csv", "wb") as file:
        file.write(response.content)

# load csv into a pandas dataframe
df_housing = pd.read_csv("./data/housing.csv")

In [4]:
df_housing

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


### Aufgabe 1

Sie haben sich sicherlich die Features im Dataframe angeschaut. Machine Learning Modelle benötigen die Daten als Zahlen um diese im Features Space abbilden zu können. Jedoch haben wir mit ocean_proximity ein Feature das Kategorische Daten enthält.

**Frage:** Um welche Art von Skalentyp handelt es sich? Wie übertragen wir ein solches Feature in einen Feature Space?

<br>
<details>
<summary><b>Lösung: Klicke hier für die Lösung.</b></summary>

Es handelt sich um eine Nominalskala. Die Ordnung ist nicht klar gegeben. Es ist z.B. nicht klar ob Island näher am Ozean ist wie Near Ocean zum Beispiel.

Diese können wir mit dem sogenannten One-Hot-Encoding in einen mathematischen Raum übertragen. Dies geschieht indem wir für jede Kategorie eine neue Dimension anlegen und dort eine 1 vermerken wenn die Kategorie zutrifft und bei allen anderen eine 0. Wir nutzen dazu den One-Hot-Encoder von Scikit-learn.

Zusätzlich entfernen wir noch alle Data Samples die leere Werte haben.

</details>


Wir listen nun einmal alle Arten von Werten die ocean_proximity haben kann.

In [ ]:
# Wir verwenden OneHotEncoder aus sklearn.preprocessing
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# TODO Konfiguriere das One-Hot-Encoding auf der Spalte 'ocean_proximity' indem du das DataFrame df_housing mit der Spaltenangabe als Parameter einfügst. Beispiel: ohe.fit(df_iris[['petal length (cm)']])
ohe.fit(...)

# Wir erstellen ein neues DataFrame mit den kodierten Spalten und füge sie dem ursprünglichen DataFrame hinzu. Danach entfernen wir die ursprüngliche Spalte 'ocean_proximity'.
df_housing_encoded = pd.concat([df_housing, pd.DataFrame(ohe.transform(df_housing[['ocean_proximity']]), columns=ohe.get_feature_names_out(['ocean_proximity']))], axis=1)
df_housing_encoded.drop('ocean_proximity', axis=1, inplace=True)

# Wir entfernen Zeilen mit fehlenden Werten, da diese nicht für das Training des Modells verwendet werden können
df_housing_encoded.dropna(inplace=True)

df_housing_encoded

### Aufgabe 2

Wir möchten nun noch die Daten normalisieren. Dies hilft einigen Modellen zum Beispiel künstlichen Neuronalen Netzwerken schneller zu optimieren und zu lernen.

Wir wenden die min-max-Skalierung an. Das heisst alle Features haben danach einen minimalen Wert von 0 und einen maximalen Wert von 1.

Wie könnten Sie dies berrechnen? Vervollständige danach den Code unten.

<details>
<summary><b>Tipp 1:</b> Klicke hier für den ersten Tipp.</summary>

Wie nutzen Sie das Minimum eines Features und das Maximum damit nachher alle Werte eines Features zwischen (inklusive) 0 und 1 sind?

</details>

<br>


<details>
<summary><b>Lösung:</b> Klicken Sie hier um die Formel anzuzeigen.</summary>

$scaled\_value = \frac{value-min}{max - min}$

</details>

In [ ]:
 # Normalisieren der numerischen Features mit Min-Max-Skalierung

# Wir haben nun eine Liste von numerischen Features
numerical_features = df_housing_encoded.select_dtypes(include=['float64', 'int64']).columns

# TODO Normalisiere die numerischen Features mit Min-Max-Skalierung
for feature in numerical_features:
    ...

df_housing_encoded

# Prüfen ob die numerischen Features korrekt normalisiert wurden
assert (df_housing_encoded[numerical_features].min().min() >= 0) and (df_housing_encoded[numerical_features].max().max() <= 1), "Die numerischen Features wurden nicht korrekt normalisiert."

### Aufgabe 3

Unterteilen Sie das Dataset in ein Trainings und Testteil wie im vorherigen Abschnitt bereits gemacht.
Nutzen Sie auch einen Train/Test Split von 80/20 und den Random State 42

In [ ]:
# Wir Unterteile das Dataset in Trainigns- und Testdaten. Die Spalte 'median_house_value' ist die Zielvariable, die wir vorhersagen möchten. 
# Deshalb wird sie von den Features getrennt. Wir entfernen die Zielvariable aus den Features bei der Parameterübergabe mit df_housing_encoded.drop('median_house_value', axis=1) und benutze sie als zweiten Parameter in der train_test_split Funktion.
# TODO fülle die fehlenden Parameter in der train_test_split Funktion aus damit 20% der Daten als Testdaten verwendet werden.

X_housing_train, X_housing_test, y_housing_train, y_housing_test = train_test_split(df_housing_encoded.drop('median_house_value', axis=1), df_housing_encoded['median_house_value'], test_size=..., random_state=42)


In [ ]:
#TODO diese Tests laufen lassen, um zu prüfen ob die Aufteilung korrekt ist
assert X_housing_train.shape[0] == 16346, f"Erwartete Anzahl Trainingsdaten: 16346, aktuell sind es: {X_housing_train.shape[0]}"
assert X_housing_test.shape[0] == 4087 , f"Erwartete Anzahl Testdaten: 4087, aktuell sind es: {X_housing_test.shape[0]}"

# Prüfe ob median_house_value aus den Features entfernt wurde
assert 'median_house_value' not in X_housing_train.columns, "median_house_value wurde nicht aus den Features entfernt."

### Aufgabe 4

1. Nutzen Sie die MLPRegressor Klasse um ein Modell zu instantieren. Die Klasse wurde bereits am Anfang importiert. Sie können die gleichen Parameter verwenden wie in Aufgabe 7 beim MLPClassifier.
2. Trainieren Sie nun das Modell mit dem Aufruf der fit(Trainingsdaten, Targets) Methode.

Optional: Weitere Infos zur MLPRegressor Klasse: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor

In [ ]:
# Wir erstellen nun ein Regressionsmodell bestehend aus mehreren Perzeptronen (MLPRegressor)
mlp_regressor = MLPRegressor(hidden_layer_sizes=(10,), max_iter=500, random_state=42)

# TODO Trainieren Sie das Modell mit den Trainignsdaten als erstes Argument und den zugehörigen Labels als zweites Argument. Tipp: benutzen Sie die X_housing_train und y_housing_train Variablen.
mlp_regressor.fit(..., ...)

### Aufgabe 5

Evaluieren Sie nun ihr Modell mit den Testdaten. Dieses Mal können wir aber nicht die Accuracy nutzen, da diese nur für Klassifikationen geeignet ist.
Wir nutzen stattdessen den Root-Mean-Squared-Error. Dieser wird wie folgt berechnet:

- $y$: Echtes Label
- $\hat{y}$: Voraussage des Modells

$\text{RMSE} = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2}$

In Prosa geschieht hier folgendes:
Für jedes Data Samples im Testdatenset wird das echte Label minus der Voraussage gerechnet. Dieses Ergebnis wird quadriert. Danach wird die Summe über alle diese quadrierten "Fehler" berechnet und geteilt durch die Anzahl Samples gerechnet. Somit der Mittelwert des quadrierten Fehlers. Zuletzt ziehen wir noch die Wurzel damit das Ergebnis besser interpretierbar wird, bezüglich der Grössenordnung.

Vervollständigen Sie den Code um den MSE zu berechnen.

In [ ]:
# Wir berechnen nun mit dem Modell die Vorhersagen für die Testdaten als einzigen Parameter in der predict-Methode
y_housing_pred = mlp_regressor.predict(X_housing_test)

# Wir berechnen den Root Mean Squared Error (RMSE) auf dem Testset
mse_test = np.sqrt(np.sum((y_housing_pred - y_housing_test)**2) / len(y_housing_test))
print(f'Mean Squared Error on Test Set: {mse_test:.4f}')

assert abs(mse_test - 0.0197) < 0.01, "Der Mean Squared Error auf dem Testset ist nicht korrekt berechnet."

### Aufgabe 6
Führen Sie die Code-Zelle unten aus. Dabei wird für ein Datasample aus dem Test Dataset der Hauspreis berechnet.

Was fällt Ihnen bei dieser Vorhersage auf?



Weshalb ist die Vorhersage in dieser Grössenordnung und wie könnten Sie dieses Problem lösen?

<br>
<details>
<summary><b>Lösung: Klicke hier für die Lösung</b>.</summary>

Wir haben auch den House Value mit Min Max Normalisierung skaliert. Wie könnte man dies nun zu einem korrekten Hauswert zurückrechnen?

</details>



In [ ]:
# Beispiel Vorhersage des Preises für ein einzelnes Haus aus dem Testset

y_housing_pred_single = mlp_regressor.predict(X_housing_test[:1])
print(f'Der berechnete Hauswert beträgt: {y_housing_pred_single[0]:.2f}')

# TODO: Optionale Zusatzaufgabe: Skalieren Sie die Vorhersage zurück in den Originalmassstab. Sie können auf die Originalen min und max Werte der Spalte 'median_house_value' im ursprünglichen DataFrame df_housing zugreifen.
#y_pred_scaled = ...
#print(f'Der berechnete Hauswert im Originalmaßstab beträgt: {y_pred_scaled[0]:.2f}')


### Aufgabe 7

Wir zeigen nun in einem Scatter Plot noch einige zufällige Datenpunkte an, wobei wir vergleichen möchten was der echte Hauspreis ist und was unser Modell berechnet hat.
Lassen Sie die nächste Code Zelle laufen und beantworten Sie die folgende Frage.

**Frage**
Woran erkennt man einen kleinen Fehler des Modells und wie einen grossen?


In [6]:
# Plotte die Vorhersagen des Modells gegen die tatsächlichen Werte nutze aber nur 50 zufällige Datenpunkte und zeichne den Fehler als Linie ein

random_indices = np.random.choice(len(y_housing_test), size=50, replace=False)
y_housing_pred_sampled = y_housing_pred[random_indices]
y_housing_test_sampled = y_housing_test.iloc[random_indices]


plt.figure(figsize=(15, 6))
plt.scatter(range(len(y_housing_pred_sampled)), y_housing_pred_sampled, color='red', label='Berechnete Werte')
plt.scatter(range(len(y_housing_test_sampled)), y_housing_test_sampled, color='blue', label='Tatsächliche Werte')
for i in range(len(y_housing_pred_sampled)):
    plt.plot([i, i], [y_housing_pred_sampled[i], y_housing_test_sampled.iloc[i]], color='gray', linestyle='--', linewidth=0.5)
plt.xlabel('Testdaten Index')
plt.ylabel('Median Hauswert (normalisiert)')
plt.title('Vorhersagen vs Tatsächliche Werte des Hauswerts')
plt.legend()
plt.show()

NameError: name 'y_housing_test' is not defined

### Zusatzaufgabe: Teste das Training ohne min-max Normalisierung

Führe nochmals einen Traingslauf durch ohne, dass die min-max Skalierung genutzt wurde.
Beobachte wie lange das Training nun läuft. 

**Frage**: Was ist schneller? Min-Max normalisierte Daten oder die ursprünglichen Daten?


## Kontrollfragen: Regression



**Kontrollfrage 3**

Was ist der Output einer Regression und wie verhält sich dieser im Vergleich zu der Klassifikation?



**Kontrollfrage 4**

Welchen Vorteil hat die Normalisierung der numerischen Features gebracht? Wie lautete die Formel?


## Zusatzaufgabe TEIL B: Klassifikation von Iris Blumen


In dieser Aufgabe sehen wir uns das Iris Dataset an. Hierbei geht es darum die Blumen anhand ihrer Blütenblätter- (Petal) und Kelchblättermasse (Sepal) zu klassifizieren.

Wir importieren zuerst einmal einige Libraries die wir nutzen möchten.

<div style="padding: 5px; border: 5px solid #a10000ff;">

**Hinweis:** In den Codezellen sind jeweils einige Codeteile nicht programmiert. Diesen Code müssen Sie ergänzen. Die jeweiligen Stellen sind mit einem Kommentar und dem Keyword **TODO** vermerkt und z.T. Stellen mit ... markiert.

Ausserdem gibt es einige assert Statements. Diese geben einen Fehler aus, sollte etwas bei Ihrer Programmierung nicht korrekt sein.

Nun laden wir das Iris Dataset, welches bereits in der Library angeboten wird. Wir speichern es auch in einem DataFrame

In [ ]:
#load iris dataset
iris = datasets.load_iris()

#create pandas dataframe from iris
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['target'] = iris.target
df_iris['target_names'] = df_iris['target'].apply(lambda x: iris.target_names[x])

Eine Aufgabe, die von Machine Learning übernommen wird, ist die Klassifizierung. 

Dabei ist es die Aufgabe ein **Data Sample** (z.B. eine Katze oder einen Hund) aufgrund von bestimmten **Features** (z.B. Anzahl Streifen, Grösse) einer bestimmten Kategorie auch **Klasse** genannt zuzuweisen. Die einzelnen Data Samples sind jeweils mit einem **Label** gekennzeichnet, zu welcher Klasse sie gehören.

### Aufgabe 8
Zeigen Sie nun das Pandas Dataframe mit den Daten an.

**Fragen**

Was sind die Features die wir in diesem Beispiel nutzen?


Was beinhaltet die Spalte target bzw. target_names?


In [ ]:
#TODO Zeige hier das Pandas DataFrame an


### Aufgabe 9
Wir zeigen nun die paarweise Kombination von den Features an. Führen Sie dazu die nächste Code-Zelle aus.

**Frage**

Wenn Sie die Blumen von Auge anhand eines der Diagramme unterscheiden müssten.
Welche Feature Kombination würde sich gut eignen? Begründen Sie.


In [ ]:
#plot the feature spaces with two features each
pairplt = sns.pairplot(df_iris.drop('target', axis=1), hue='target_names', height=2, palette='tab10')


### Aufgabe 10
In diesem Fall möchten wir nun die Blumen in die Klassen setosa (0), versicolor (1) und verginica (2) einteilen. Die Daten haben alle bereits die sogenannten **Labels** zugewiesen. Diese werden manchmal auch target genannt und sind in den Daten in den Spalten target und target_names vorhanden.

Wenn wir ein oder mehrere Features in einem mathematischen Raum kombinieren, entsteht ein sogennanter **Feature Space** (Merkmalsraum).
Ein solcher Feature Space für die *petal width* und *petal length* sieht wie folgt aus. 

In [ ]:
# TODO Vervollständigen Sie die Spaltennamen der Parameter x und y für einen Scatterplot der Petal Length gegen die Petal Width

sns.scatterplot(x='...', y='...',
                hue='target_names', data=df_iris.drop('target', axis=1), palette='tab10')

# Placing Legend outside the Figure
plt.legend(bbox_to_anchor=(1, 1), loc=2)
plt.show()


**Fragen**

Wie viele Dimensionen hat der Feature Space im obigen Beispiel?


Wie viele Dimensionen können wir in einem Feature Space haben?


### Aufgabe 11

Ein Machine Learning Model versucht eine oder mehrere **Decision Boundaries** also Entscheidungsgrenzen in den Feature Space zu platzieren, so dass möglichst viele Data Samples korrekt klassifiziert werden.

**Frage**

Beschreiben Sie wie Sie zwei linearen Decision Boundaries im Feature Space oben platzieren würden um die drei Klassen auseinanderzuhalten.



### Aufgabe 12

Machine Learning Modelle treffen Entscheidungen anhand von Daten, welche Sie zu Beginn der «Kalibrierung» genutzt haben, um zu lernen. Man nennt diese Daten **Trainingsdaten**, da das Modell sozusagen damit trainiert wird. Wenn nun ein Machine Learning Modell getestet wird (heisst: wie gut funktioniert das Modell?), werden ihm neue Daten sogenannte **Testdaten** präsentiert, welche nicht für das Training genutzt wurden. 

**Fragen**

Was ist der Vorteil bei diesem Vorgehen? 


Was könnte ein Nachteil sein?



### Aufgabe 13

Damit wir Trainings- und Testdaten haben, teilen wir die Daten in Trainings und Testdaten ein. Das gleiche passiert auch mit den targets. Wir nutzen eine fertige Funktion von sklearn dafür.

Vielfach werden die Features in einer Variable X gespeichert und die Labels bzw. Targets in einer Variable y.

Wir machen das gleich und kennzeichnen gleich im Variablennamen auch ob die Daten die Trainings oder Testdaten sind.

**Fragen**

In welchem Verhältnis werden die Daten mit dem Befehl train_test_split unten aufgeteilt?


Wie viele Data Samples sind nun im Trainings Dataset und wie viele im Testdataset?


In [ ]:
X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

#TODO Ausgabe der Anzahl der Trainings- und Testdaten


### Aufgabe 14

Wir trainieren nun unseren ersten Classifier und testen diesen auch. 

Vorerst nutzen wir einen fixfertigen Classifier von Sklearn den wir instanziieren müssen und dann "fitten". Hinter der fit Funktion versteckt sich ein Trainingsprozess in dem das Modell von den Trainingsdaten lernt.
Vervollständige die Befehle anhand der Kommentare.

**Bemerkung:** Das Modell wurde extra so konfiguriert, dass nicht eine 100% Genauigkeit resultiert, damit wir die Genauigkeit auswerten können. Deshalb erscheint auch eine **ConvergenceWarning**, diese können Sie **ignorieren**.

Wir evaluieren das Modell mittels der Accuracy. Diese wird berechnet indem die Anzahl korrekt klassifizierten Data Samples geteilt durch die Anzahl aller Data Samples gerechnet wird:

  $ Accuracy = \frac{\text{Anzahl korrekt Klassifizierte Data Samples}}{\text{Alle Data Samples}} $


In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(10,), max_iter=500, random_state=42)

#TODO Trainiere das Modell mit den Trainingsdaten als erstes Argument und den zugehörigen Labels als zweites Argument
mlp.fit(..., ...)

#TODO Berechne nun mit dem Modell die Klassen für die Testdaten als einzigen Parameter in der predict-Methode
y_iris_pred = mlp.predict(...)

# TODO Berechne die Genauigkeit des Modells
# Dies können wir berechnen indem wir zählen, wie viele der vorhergesagten Labels mit den tatsächlichen Labels übereinstimmen und dies durch die Gesamtanzahl der Testdaten teilen
accuracy = ...

# Test ob die Accuracy korrekt berechnet wurde
assert (abs(accuracy-0.93) < 0.01), "Die Genauigkeit des Modells ist nicht korrekt berechnet."
print(f'Accuracy: {accuracy*100:.2f}%')

### Aufgabe 8

Wir schauen uns nun noch genauer an, welche Klassen am meisten verwechselt wurden. Dazu werden wir eine Konfusionsmatrix nutzen. Dabei werden die tatsächlichen Labels den vorhergesagten Labels gegenübergestellt und aufgezeichnet, wie oft jede Kombination vorkommt.


**Frage**
Betrachten Sie nun nochmals die paarweisen Feature Spaces aus Aufgabe 2 betrachten.
Welche zwei Klassen werden wohl am meisten verwechselt?


Lassen Sie nun die Zelle unten laufen und interpretieren Sie die Konfusionsmatrix. Stimmte ihre Vermutung?



In [ ]:
# Wir zeigen nun eine Konfusionsmatrix an, um die Leistung des Modells zu visualisieren. 

conf_matrix = ConfusionMatrixDisplay.from_predictions(y_iris_test, y_iris_pred, display_labels=iris.target_names)
conf_matrix.ax_.set_title('Confusion Matrix')
plt.show()

## Kontrollfragen Klassifikation

**Kontrollfrage 1**

Ein Machine-Learning Modell klassifiziert die Schüler*innen ob sie mit dem Velo, den ÖV oder zu Fuss zur Schule kommen. 
In den Daten wird pro Schüler*in und Tag erfasst, wie weit der Schulweg ist, wie lange die Reisedauer war, ob der/die Schüler*in ein Velo besitzt und wie weit die nächste ÖV-Haltestelle vom Wohnort entfernt ist.

Was sind in diesem Beispiel

    - Features?
    - Klassen?
    - Data Samples?
    
Was wäre ein Beispiel für eine Ein und Ausgabe des Modells?






**Kontrollfrage 2**

Beschreiben Sie das Vorgehen, um ein Klassifikationsmodell zu evaluieren und dabei die Accuracy zu berechnen.

